# Importacion de API key

In [28]:
import os
import sys
import json
import math
import time
import numpy as np
import pandas as pd
import requests
import folium
from datetime import datetime, timezone

# 1) Calculamos la ruta absoluta de la carpeta “proyecto”
#    Partimos de que os.getcwd() = .../proyecto/notebooks
ruta_notebook = os.getcwd()
ruta_proyecto = os.path.abspath(os.path.join(ruta_notebook, ".."))

In [14]:
# 1) Calculamos la ruta absoluta de ../src respecto al notebook
ruta_src = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))

# 2) Añadimos esa ruta al sys.path (si no está ya)
if ruta_src not in sys.path:
    sys.path.append(ruta_src)

# 3) Ahora sí podemos importar
from config import API_KEY_OWM, API_KEY_SG

In [15]:
# Comprobamos que funcionan (puedes imprimirlas)
print("OWM key:", API_KEY_OWM)
print("SG  key:", API_KEY_SG)

OWM key: 0b87f738ea444ed359d9338b6fab8ecb
SG  key: b73c5ca4-3e4e-11f0-976d-0242ac130006-b73c5d4e-3e4e-11f0-976d-0242ac130006


# Definir la zona geográfica y la resolución de la malla. 

## Mapa para seleccionar zona

In [5]:
# — Celda 1: Mostrar el mapa con Draw (solo rectángulo) —
import folium
from folium.plugins import Draw

lat, lon, zoom = 40.35, 0.40, 10
m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles='OpenStreetMap')

folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=1.0
).add_to(m)

draw = Draw(
    export=True,
    filename='selection.geojson',
    position='topright',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'marker': False,
        'circlemarker': False,
        'rectangle': True,
    },
    edit_options={'edit': False}
)
draw.add_to(m)

folium.LayerControl().add_to(m)
m


## Leer zona seleccionada y establecer limites

In [10]:
# 2) Construimos la ruta absoluta hasta data/raw/selection.geojson
path_geojson = os.path.join(ruta_proyecto, "data", "raw", "selection.geojson")


In [11]:
# 3) Comprobamos (opcional) que el fichero existe
if not os.path.isfile(path_geojson):
    raise FileNotFoundError(f"No se encontró el fichero en:\n{path_geojson}")

In [12]:
# 4) Abrimos y leemos el GeoJSON
with open(path_geojson, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 5) Extraemos los límites del rectángulo
coords = data['features'][0]['geometry']['coordinates'][0]
lon1, lat1 = coords[0]
lon2, lat2 = coords[2]

lat_min = min(lat1, lat2)
lat_max = max(lat1, lat2)
lon_min = min(lon1, lon2)
lon_max = max(lon1, lon2)

print("Área de estudio:")
print(f"  lat_min = {lat_min:.6f}, lat_max = {lat_max:.6f}")
print(f"  lon_min = {lon_min:.6f}, lon_max = {lon_max:.6f}")


Área de estudio:
  lat_min = 38.753227, lat_max = 40.663704
  lon_min = 1.980978, lon_max = 4.819924


## Generación de malla

In [17]:
# (a) Paso en longitud
paso_lon = 0.1  # ajusta 0.05, 0.02, etc. si quieres otra resolución
N_lon = int(np.floor((lon_max - lon_min) / paso_lon))
lon_max_ajustada = lon_min + N_lon * paso_lon
lons = np.linspace(lon_min, lon_max_ajustada, N_lon + 1)

# (b) Proyección Mercator para lat
phi1 = math.radians(lat_min)
phi2 = math.radians(lat_max)
y1 = math.log(math.tan(math.pi/4 + phi1/2))
y2 = math.log(math.tan(math.pi/4 + phi2/2))

# Estimación de Δy que aprox. equivale a 0.1° en latitud media
lat_media = 0.5 * (lat_min + lat_max)
delta_phi_rads = math.radians(0.1)
y_media = math.log(math.tan(math.pi/4 + math.radians(lat_media)/2))
y_punto_arriba   = math.log(math.tan(math.pi/4 + (math.radians(lat_media) + delta_phi_rads)/2))
delta_y_aprox = abs(y_punto_arriba - y_media)

# Número de intervalos en y
N_lat = int(np.floor((y2 - y1) / delta_y_aprox))
delta_y_exacto = (y2 - y1) / N_lat

# Generar array de ys uniformes
ys = np.array([y1 + i*delta_y_exacto for i in range(N_lat+1)])
# Invertir a latitudes en radianes y luego a grados
lats = np.array([(2*math.atan(math.exp(y))) - math.pi/2 for y in ys])
lats = np.degrees(lats)

# —————————————————————————————————————————————————————
# 3) Estado final de la malla
# —————————————————————————————————————————————————————
print(f"Número de nodos en latitud:  {len(lats)}")
print(f"Número de nodos en longitud: {len(lons)}")
print(f"Total nodos (malla): {len(lats)*len(lons)}")

# 4) (Opcional) Lista de tuplas (lat, lon) para iterar
puntos_malla = [(lat, lon) for lat in lats for lon in lons]

Número de nodos en latitud:  20
Número de nodos en longitud: 29
Total nodos (malla): 580


### Mostrar malla con capas náuticas

In [18]:
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

# Capa de símbolos náuticos (OpenSeaMap)
folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

# Capa de contornos batimétricos (EMODnet)
folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=0.7
).add_to(m)

# Dibujar cada nodo como un CircleMarker
for lat in lats:
    for lon in lons:
        folium.CircleMarker(
            location=(lat, lon),
            radius=2,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            weight=0
        ).add_to(m)

# Control de capas
folium.LayerControl().add_to(m)

# Mostrar el mapa (en un notebook se renderiza automáticamente)
m


## Calculo de pesos

### Funciones de consulta a api

In [29]:

def iso_to_unix(fecha_iso: str) -> int:
    """
    Convierte un string ISO 8601 a timestamp UNIX (segundos).
    """
    dt = datetime.fromisoformat(fecha_iso.replace("Z", "+00:00"))
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    dt_utc = dt.astimezone(timezone.utc)
    return int(dt_utc.timestamp())


def get_owm_wind_historical(lat: float, lon: float, fecha_iso: str, api_key: str) -> dict:
    """
    Llama a History API de OpenWeatherMap para obtener la velocidad y dirección
    del viento en el instante especificado (fecha_iso).
    """
    # 1) Convertir fecha ISO a UNIX
    t_unix = iso_to_unix(fecha_iso)
    start = t_unix
    end = t_unix + 3599  # bloque horario completo

    url = "https://history.openweathermap.org/data/2.5/history/city"
    params = {
        "lat":   lat,
        "lon":   lon,
        "type":  "hour",
        "start": start,
        "end":   end,
        "appid": api_key
    }
    r = requests.get(url, params=params)
    if r.status_code != 200:
        # Si no hay datos históricos o la suscripción no lo permite, devolvemos NaN
        return {"owm_wind_speed": float("nan"), "owm_wind_deg": float("nan")}

    data = r.json()
    if "list" not in data or len(data["list"]) == 0:
        # No hay datos en ese intervalo
        return {"owm_wind_speed": float("nan"), "owm_wind_deg": float("nan")}

    bloque = data["list"][0]
    wind = bloque.get("wind", {})
    speed = wind.get("speed", float("nan"))
    deg   = wind.get("deg",   float("nan"))
    return {"owm_wind_speed": speed, "owm_wind_deg": deg}


def get_stormglass_all(lat: float, lon: float, fecha_iso: str, api_key: str) -> dict:
    """
    Consulta StormGlass para viento, olas y corrientes en un punto (lat, lon)
    en el instante fecha_iso (start=end=fecha_iso).
    """
    url = "https://api.stormglass.io/v2/weather/point"
    headers = {"Authorization": api_key}
    params = {
        "lat": lat,
        "lng": lon,
        "params": ",".join([
            "windSpeed", "windDirection",
            "waveHeight", "waveDirection",
            "currentSpeed", "currentDirection"
        ]),
        "start": fecha_iso,
        "end":   fecha_iso
    }
    r = requests.get(url, params=params, headers=headers)
    if r.status_code != 200:
        return {
            "sg_wind_speed":    float("nan"),
            "sg_wind_dir":      float("nan"),
            "sg_wave_height":   float("nan"),
            "sg_wave_dir":      float("nan"),
            "sg_current_speed": float("nan"),
            "sg_current_dir":   float("nan")
        }
    data = r.json()
    if "hours" not in data or len(data["hours"]) == 0:
        return {
            "sg_wind_speed":    float("nan"),
            "sg_wind_dir":      float("nan"),
            "sg_wave_height":   float("nan"),
            "sg_wave_dir":      float("nan"),
            "sg_current_speed": float("nan"),
            "sg_current_dir":   float("nan")
        }
    hour0 = data["hours"][0]
    return {
        "sg_wind_speed":    hour0["windSpeed"]["noaa"],
        "sg_wind_dir":      hour0["windDirection"]["noaa"],
        "sg_wave_height":   hour0["waveHeight"]["noaa"],
        "sg_wave_dir":      hour0["waveDirection"]["noaa"],
        "sg_current_speed": hour0["currentSpeed"]["noaa"],
        "sg_current_dir":   hour0["currentDirection"]["noaa"]
    }

### Recorrer malla añadiendo datos "instante puntual"

In [ ]:
fecha_iso = "2024-06-01T00:00:00+00:00"
puntos_malla = [(lat, lon) for lat in lats for lon in lons]

filas = []
for idx, (lat, lon) in enumerate(puntos_malla):
    try:
        # Obtener viento histórico de OWM
        owm = get_owm_wind_historical(lat, lon, fecha_iso, API_KEY_OWM)

        # Solo guardamos columnas de OpenWeatherMap; dejamos todos los campos de SG como NaN
        fila = {
            "lat": lat,
            "lon": lon,
            "owm_wind_speed": owm["owm_wind_speed"],
            "owm_wind_deg":   owm["owm_wind_deg"],
            # Campos de StormGlass puestos a NaN por ahora
            "sg_wind_speed":    float("nan"),
            "sg_wind_dir":      float("nan"),
            "sg_wave_height":   float("nan"),
            "sg_wave_dir":      float("nan"),
            "sg_current_speed": float("nan"),
            "sg_current_dir":   float("nan")
        }
    except Exception as e:
        # En caso de error, registramos NaN en todas las variables
        fila = {
            "lat": lat,
            "lon": lon,
            "owm_wind_speed":   float("nan"),
            "owm_wind_deg":     float("nan"),
            "sg_wind_speed":    float("nan"),
            "sg_wind_dir":      float("nan"),
            "sg_wave_height":   float("nan"),
            "sg_wave_dir":      float("nan"),
            "sg_current_speed": float("nan"),
            "sg_current_dir":   float("nan")
        }
        print(f"Error nodo ({lat:.4f}, {lon:.4f}): {e}")

    filas.append(fila)
    time.sleep(0.2)  # pequeña pausa para no saturar la API

Error nodo (38.7532, 1.9810): 'noaa'
Error nodo (38.7532, 2.0810): 'noaa'
Error nodo (38.7532, 2.1810): 'noaa'
Error nodo (38.7532, 2.2810): 'noaa'
Error nodo (38.7532, 2.3810): 'noaa'


KeyboardInterrupt: 

#### Proveedores de los dos wether api

In [31]:
import requests
import os, sys

# (1) Ajustamos path para importar config.py si hace falta
# … tu código para ajustar sys.path …

from config import API_KEY_SG

def ver_proveedores_stormglass(lat, lon, fecha_iso, api_key):
    """
    Imprime los proveedores disponibles para windSpeed, waveHeight, currentSpeed, 
    etc., en el primer bloque horario de la respuesta de StormGlass.
    """
    url = "https://api.stormglass.io/v2/weather/point"
    headers = {"Authorization": api_key}
    params = {
        "lat": lat,
        "lng": lon,
        "params": "windSpeed,waveHeight,currentSpeed",
        "start": fecha_iso,
        "end":   fecha_iso
    }
    r = requests.get(url, params=params, headers=headers)
    if r.status_code != 200:
        print(f"Error en la llamada a StormGlass ({r.status_code}): {r.text}")
        return

    data = r.json()
    if "hours" not in data or len(data["hours"]) == 0:
        print("No hay datos en 'hours'")
        return

    primer_bloque = data["hours"][0]
    for parametro in ["windSpeed", "windDirection", "waveHeight", "waveDirection", "currentSpeed", "currentDirection"]:
        dic = primer_bloque.get(parametro, {})
        if isinstance(dic, dict) and len(dic) > 0:
            print(f"Proveedores para {parametro}: {list(dic.keys())}")
        else:
            print(f"No hay datos para {parametro}")

# Ejemplo de uso:
lat_test, lon_test = 39.0, 3.0
fecha_iso = "2024-06-01T00:00:00+00:00"
ver_proveedores_stormglass(lat_test, lon_test, fecha_iso, API_KEY_SG)


Proveedores para windSpeed: ['icon', 'noaa', 'sg']
No hay datos para windDirection
Proveedores para waveHeight: ['dwd', 'icon', 'meteo', 'noaa', 'sg']
No hay datos para waveDirection
Proveedores para currentSpeed: ['meto', 'sg']
No hay datos para currentDirection


In [32]:
import requests
import os
import sys

# (1) Ajustamos sys.path para poder importar config.py
ruta_notebook = os.getcwd()  # Asumiendo que ejecutas desde ./notebooks
ruta_proyecto = os.path.abspath(os.path.join(ruta_notebook, ".."))
ruta_src = os.path.join(ruta_proyecto, "src")
if ruta_src not in sys.path:
    sys.path.append(ruta_src)

from config import API_KEY_SG

def ver_proveedores_stormglass(lat, lon, fecha_iso, api_key):
    """
    Imprime los proveedores disponibles para windSpeed, windDirection,
    waveHeight, waveDirection, currentSpeed y currentDirection
    en el primer bloque horario de la respuesta de StormGlass.
    """
    url = "https://api.stormglass.io/v2/weather/point"
    headers = {"Authorization": api_key}
    params = {
        "lat": lat,
        "lng": lon,
        "params": ",".join([
            "windSpeed", "windDirection",
            "waveHeight", "waveDirection",
            "currentSpeed", "currentDirection"
        ]),
        "start": fecha_iso,
        "end":   fecha_iso
    }
    r = requests.get(url, params=params, headers=headers)
    if r.status_code != 200:
        print(f"Error en la llamada a StormGlass ({r.status_code}): {r.text}")
        return

    data = r.json()
    if "hours" not in data or len(data["hours"]) == 0:
        print("No hay datos en 'hours'")
        return

    primer_bloque = data["hours"][0]
    for parametro in [
        "windSpeed", "windDirection",
        "waveHeight", "waveDirection",
        "currentSpeed", "currentDirection"
    ]:
        dic = primer_bloque.get(parametro, {})
        if isinstance(dic, dict) and len(dic) > 0:
            print(f"Proveedores para {parametro}: {list(dic.keys())}")
        else:
            print(f"No hay datos para {parametro}")

# Ejemplo de uso:
lat_test, lon_test = 39.0, 3.0
fecha_iso = "2024-06-01T00:00:00+00:00"
ver_proveedores_stormglass(lat_test, lon_test, fecha_iso, API_KEY_SG)


Proveedores para windSpeed: ['icon', 'noaa', 'sg']
Proveedores para windDirection: ['icon', 'noaa', 'sg']
Proveedores para waveHeight: ['dwd', 'icon', 'meteo', 'noaa', 'sg']
Proveedores para waveDirection: ['icon', 'meteo', 'noaa', 'sg']
Proveedores para currentSpeed: ['meto', 'sg']
Proveedores para currentDirection: ['meto', 'sg']


### Generar data frame

In [ ]:
df = pd.DataFrame(filas)
print(df.head())

# Si quieres guardar a CSV:
ruta_salida = os.path.join(ruta_proyecto, "data", "processed", "malla_instante_20240601.csv")
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
df.to_csv(ruta_salida, index=False)
print(f"Datos guardados en: {ruta_salida}")